<a href="https://colab.research.google.com/github/samer-glitch/TADP-Enhancing-Trustworthiness-in-Federated-Learning-Through-Provenance-Driven-Governance/blob/main/Governance_%2B_Fashion_MNIST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
"""
Sequential Federated Learning with Governance — Fashion-MNIST (Medium Pipeline)

Taxonomy:

  No governance:
    - FedAvg
    - FedProx

  Robust FL:
    - Median
    - Krum

  Data-driven selection:
    - Power-of-Choice (top-k based on local acc)

  Soft / hard governance:
    - PScoreWeightedFedAvgAll       (all clients, PScore-weighted FedAvg)
    - PScoreWeightedFedAvgWarmup    (warm-up + PScore-weighted)
    - StaticPScoreGate              (hard gate + PScore-weighted FedAvg)
    - DynamicPScoreV3               (warm-up + soft down-weighting, no filtering)

Outputs per scenario:
    - Final Accuracy, F1, Recall, AUC-ROC (macro)
    - Total Training Time (s)
    - Total Energy (kWh)
    - Total CO2 (kg)
    - Total Communication (MB)
    - Selected Clients, Trained Clients
    - Avg PScore Selected
"""

import os
import time
import random
from typing import List, Dict, Tuple

import numpy as np
import pandas as pd

from sklearn.metrics import (
    accuracy_score,
    f1_score,
    recall_score,
    roc_auc_score,
)

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

from torchvision import datasets, transforms

# ============================================================
#  GLOBAL CONFIG (FAST VERSION)
# ============================================================

RANDOM_STATE = 42

# Fast config for FMNIST
N_CLIENTS = 6
N_ROUNDS = 5           # reduced rounds
LOCAL_EPOCHS = 1       # single local epoch
BATCH_SIZE = 32
LR = 1e-3

ALPHA_DIRICHLET = 0.5  # non-IID strength

DEVICE_POWER_KW = 0.1
CO2_PER_KWH = 0.4
MODEL_SIZE_MB = 0.5    # small CNN, lower comms

# Robust / selection params
POC_TOP_K = 3           # Power-of-Choice (top-k by local acc)
KRUM_F_BYZANTINE = 1    # assumed Byzantine clients for Krum

# Static PScore gate
GOVERNANCE_PERCENTILE = 30.0  # 30th percentile threshold

# Dynamic PScore-v3 (soft down-weighting)
DYN_WARMUP_ROUNDS = 2   # fewer warm-up rounds for speed
DYN_ALPHA_PROV = 0.7
DYN_LOW_PENALTY = 0.5

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")


# ============================================================
#  UTILS
# ============================================================

def set_global_seed(seed: int = RANDOM_STATE):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)


def estimate_energy_kwh(t_s: float) -> float:
    return DEVICE_POWER_KW * (t_s / 3600.0)


def estimate_co2_kg(kwh: float) -> float:
    return kwh * CO2_PER_KWH


# ============================================================
#  DATA LOADING — FASHION-MNIST
# ============================================================

def load_fmnist() -> Tuple[np.ndarray, np.ndarray, np.ndarray, np.ndarray]:
    """
    Returns:
        X_train: (N_train, 1, 28, 28), float32 in [0,1]
        X_test:  (N_test, 1, 28, 28), float32 in [0,1]
        y_train, y_test: int64 labels [0..9]
    """
    print("📊 Loading Fashion-MNIST...")

    train_set = datasets.FashionMNIST(
        root="./data",
        train=True,
        download=True,
        transform=None
    )
    test_set = datasets.FashionMNIST(
        root="./data",
        train=False,
        download=True,
        transform=None
    )

    # train_set.data: (60000, 28, 28), uint8
    X_train = train_set.data.numpy().astype(np.float32) / 255.0
    X_train = np.expand_dims(X_train, 1)  # (N, 1, 28, 28)
    y_train = train_set.targets.numpy().astype(np.int64)

    X_test = test_set.data.numpy().astype(np.float32) / 255.0
    X_test = np.expand_dims(X_test, 1)
    y_test = test_set.targets.numpy().astype(np.int64)

    print(f"✅ F-MNIST loaded: train={X_train.shape[0]}, test={X_test.shape[0]}")
    return X_train, X_test, y_train, y_test


# ============================================================
#  NON-IID DIRICHLET CLIENT SPLITTER
# ============================================================

def create_dirichlet_clients(
    X: np.ndarray,
    y: np.ndarray,
    n_clients: int,
    alpha: float,
) -> List[Dict]:

    print(f"🔧 Creating Non-IID clients (Dirichlet α={alpha})")
    n_classes = len(np.unique(y))
    idx_by_class = [np.where(y == k)[0] for k in range(n_classes)]

    client_indices = [[] for _ in range(n_clients)]

    for k in range(n_classes):
        idx_k = idx_by_class[k]
        np.random.shuffle(idx_k)

        proportions = np.random.dirichlet(alpha * np.ones(n_clients))
        counts = (proportions * len(idx_k)).astype(int)

        diff = len(idx_k) - counts.sum()
        for i in range(abs(diff)):
            counts[i % n_clients] += int(np.sign(diff))

        start = 0
        for cid in range(n_clients):
            end = min(start + counts[cid], len(idx_k))
            client_indices[cid].extend(idx_k[start:end])
            start = end

    clients = []
    for cid in range(n_clients):
        idx = np.array(client_indices[cid])
        Xc, yc = X[idx], y[idx]
        dist = dict(zip(*np.unique(yc, return_counts=True)))
        print(f"   Client {cid}: {len(yc)} samples → {dist}")
        clients.append({"id": cid, "X": Xc, "y": yc})

    print("✅ Clients created.\n")
    return clients


# ============================================================
#  MODEL — SMALL CNN FOR FASHION-MNIST
# ============================================================

class FMNISTNet(nn.Module):
    def __init__(self, n_classes: int = 10):
        super().__init__()
        self.features = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),

            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2),   # 14x14

            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2),   # 7x7
        )
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(128 * 7 * 7, 256),
            nn.ReLU(inplace=True),
            nn.Linear(256, n_classes),
        )

    def forward(self, x):
        x = self.features(x)
        x = self.classifier(x)
        return x


def init_global_model() -> nn.Module:
    return FMNISTNet(n_classes=10).to(DEVICE)


def get_model_params(model: nn.Module) -> Dict[str, torch.Tensor]:
    return {k: v.detach().cpu().clone() for k, v in model.state_dict().items()}


def set_model_params(model: nn.Module, params: Dict[str, torch.Tensor]):
    model.load_state_dict(params, strict=True)


def average_state_dicts(
    param_list: List[Dict[str, torch.Tensor]],
    weights: List[float],
) -> Dict[str, torch.Tensor]:
    weights = np.array(weights, dtype=float)
    weights = weights / (weights.sum() + 1e-12)

    avg = {}
    for k in param_list[0].keys():
        stacked = torch.stack([
            p[k].float() * float(w) for p, w in zip(param_list, weights)
        ])
        avg[k] = stacked.sum(dim=0)
    return avg


# ============================================================
#  LOCAL TRAINING (FEDAVG) + FEDPROX
# ============================================================

CRITERION = nn.CrossEntropyLoss()


def local_train(
    global_model: nn.Module,
    X: np.ndarray,
    y: np.ndarray,
) -> Tuple[Dict[str, torch.Tensor], float, float]:

    if X.shape[0] == 0:
        return get_model_params(global_model), 0.0, 0.0

    local_model = FMNISTNet(n_classes=10).to(DEVICE)
    local_model.load_state_dict(global_model.state_dict())

    optimizer = optim.Adam(local_model.parameters(), lr=LR)

    X_tensor = torch.from_numpy(X).float().to(DEVICE)
    y_tensor = torch.from_numpy(y.astype(np.int64)).long().to(DEVICE)

    n_samples = X_tensor.shape[0]
    indices = np.arange(n_samples)

    start = time.time()
    local_model.train()

    for _ in range(LOCAL_EPOCHS):
        np.random.shuffle(indices)
        for s in range(0, n_samples, BATCH_SIZE):
            e = min(s + BATCH_SIZE, n_samples)
            idx = indices[s:e]
            xb = X_tensor[idx]
            yb = y_tensor[idx]

            optimizer.zero_grad()
            logits = local_model(xb)
            loss = CRITERION(logits, yb)
            loss.backward()
            optimizer.step()

    train_time = time.time() - start

    local_model.eval()
    with torch.no_grad():
        logits = local_model(X_tensor)
        preds = torch.argmax(logits, dim=1).cpu().numpy()

    acc = accuracy_score(y, preds)
    return get_model_params(local_model), train_time, acc


def local_train_fedprox(
    global_model: nn.Module,
    X: np.ndarray,
    y: np.ndarray,
    mu: float,
) -> Tuple[Dict[str, torch.Tensor], float, float]:

    if X.shape[0] == 0:
        return get_model_params(global_model), 0.0, 0.0

    local_model = FMNISTNet(n_classes=10).to(DEVICE)
    local_model.load_state_dict(global_model.state_dict())

    global_params = {k: v.detach().clone() for k, v in global_model.state_dict().items()}
    optimizer = optim.Adam(local_model.parameters(), lr=LR)

    X_tensor = torch.from_numpy(X).float().to(DEVICE)
    y_tensor = torch.from_numpy(y.astype(np.int64)).long().to(DEVICE)

    n_samples = X_tensor.shape[0]
    indices = np.arange(n_samples)

    start = time.time()
    local_model.train()

    for _ in range(LOCAL_EPOCHS):
        np.random.shuffle(indices)
        for s in range(0, n_samples, BATCH_SIZE):
            e = min(s + BATCH_SIZE, n_samples)
            idx = indices[s:e]
            xb = X_tensor[idx]
            yb = y_tensor[idx]

            optimizer.zero_grad()
            logits = local_model(xb)
            base_loss = CRITERION(logits, yb)

            prox_term = 0.0
            for name, param in local_model.named_parameters():
                prox_term = prox_term + torch.sum(
                    (param - global_params[name].to(DEVICE)) ** 2
                )

            loss = base_loss + (mu / 2.0) * prox_term
            loss.backward()
            optimizer.step()

    train_time = time.time() - start

    local_model.eval()
    with torch.no_grad():
        logits = local_model(X_tensor)
        preds = torch.argmax(logits, dim=1).cpu().numpy()

    acc = accuracy_score(y, preds)
    return get_model_params(local_model), train_time, acc


# ============================================================
#  GLOBAL EVALUATION
# ============================================================

def evaluate_global_model(
    model: nn.Module,
    X_test: np.ndarray,
    y_test: np.ndarray,
) -> Dict[str, float]:

    model.eval()
    with torch.no_grad():
        X_t = torch.from_numpy(X_test).float().to(DEVICE)
        logits = model(X_t)
        probs = F.softmax(logits, dim=1).cpu().numpy()
        preds = np.argmax(probs, axis=1)

    acc = accuracy_score(y_test, preds)
    f1 = f1_score(y_test, preds, average="macro")
    rec = recall_score(y_test, preds, average="macro")

    try:
        auc = roc_auc_score(y_test, probs, multi_class="ovr")
    except ValueError:
        auc = float("nan")

    return {"accuracy": acc, "f1": f1, "recall": rec, "auc": auc}


# ============================================================
#  ROBUST AGGREGATION HELPERS (MEDIAN, KRUM)
# ============================================================

def median_aggregate(param_list: List[Dict[str, torch.Tensor]]) -> Dict[str, torch.Tensor]:
    out = {}
    for k in param_list[0].keys():
        stacked = torch.stack([p[k].float() for p in param_list])  # [C, ...]
        median_vals = torch.median(stacked, dim=0).values
        out[k] = median_vals
    return out


def _param_distance(p1: Dict[str, torch.Tensor], p2: Dict[str, torch.Tensor]) -> float:
    dist = 0.0
    for k in p1.keys():
        diff = (p1[k].float() - p2[k].float()).view(-1)
        dist += torch.sum(diff * diff).item()
    return dist


def krum_aggregate(param_list: List[Dict[str, torch.Tensor]], f_byz: int) -> Tuple[Dict[str, torch.Tensor], int]:
    """
    Classical Krum:
      - For each client update i, compute sum of distances to its
        closest (m - f_byz - 2) other updates.
      - Choose update with minimal score.
    """
    m = len(param_list)
    if m <= 2 * f_byz + 2:
        avg = average_state_dicts(param_list, [1.0] * m)
        return avg, -1

    dists = np.zeros((m, m), dtype=float)
    for i in range(m):
        for j in range(i + 1, m):
            d = _param_distance(param_list[i], param_list[j])
            dists[i, j] = dists[j, i] = d

    scores = []
    for i in range(m):
        others = np.delete(dists[i], i)
        nb_closest = m - f_byz - 2
        score_i = np.sum(np.partition(others, nb_closest)[:nb_closest])
        scores.append(score_i)

    winner = int(np.argmin(scores))
    return param_list[winner], winner


# ============================================================
#  PSCORE UTILITIES
# ============================================================

def get_static_pscores(n_clients: int) -> List[float]:
    # Same demonstration vector as Adult pipeline
    base = [
        4.05, 3.92, 3.94, 3.03, 3.28,
        2.83, 2.39, 1.58, 2.37, 1.51
    ]
    return base[:n_clients]


def compute_governance_threshold(pscores: List[float], percentile: float) -> float:
    arr = np.array(pscores, dtype=float)
    return float(np.percentile(arr, percentile))


def minmax_normalize_pscores(pscores: List[float]) -> np.ndarray:
    ps = np.array(pscores, dtype=float)
    p_min, p_max = ps.min(), ps.max()
    p_rng = p_max - p_min if p_max > p_min else 1.0
    return (ps - p_min) / p_rng


# ============================================================
#  FEDERATED SCENARIOS
# ============================================================

def run_fedavg(clients, X_test, y_test, pscores):
    print("🎯 FedAvg")

    global_model = init_global_model()
    total_t = 0.0

    for r in range(1, N_ROUNDS + 1):
        print(f"  Round {r}")
        local_params, weights = [], []
        for c in clients:
            cid = c["id"]
            params, t, acc = local_train(global_model, c["X"], c["y"])
            print(f"     Client {cid} acc={acc:.3f}")
            total_t += t
            local_params.append(params)
            weights.append(len(c["y"]))

        avg = average_state_dicts(local_params, weights)
        set_model_params(global_model, avg)
        m = evaluate_global_model(global_model, X_test, y_test)
        print(f"     🟢 Acc={m['accuracy']:.4f}")

    final = evaluate_global_model(global_model, X_test, y_test)
    energy = estimate_energy_kwh(total_t)
    co2 = estimate_co2_kg(energy)
    comm = N_ROUNDS * len(clients) * MODEL_SIZE_MB

    return {
        "Scenario": "FedAvg",
        "Final Accuracy": final["accuracy"],
        "Final F1-Score": final["f1"],
        "Final Recall": final["recall"],
        "Final AUC-ROC": final["auc"],
        "Total Training Time (s)": total_t,
        "Total Energy (kWh)": energy,
        "Total CO2 (kg)": co2,
        "Total Communication (MB)": comm,
        "Selected Clients": len(clients),
        "Trained Clients": len(clients),
        "Avg PScore Selected": float(np.mean(pscores)),
    }


def run_fedprox(clients, X_test, y_test, pscores, mu: float = 0.01):
    print("\n🎯 FedProx (mu={})".format(mu))

    global_model = init_global_model()
    total_t = 0.0

    for r in range(1, N_ROUNDS + 1):
        print(f"  Round {r}")
        local_params, weights = [], []
        for c in clients:
            cid = c["id"]
            params, t, acc = local_train_fedprox(global_model, c["X"], c["y"], mu)
            print(f"     Client {cid} acc={acc:.3f}")
            total_t += t
            local_params.append(params)
            weights.append(len(c["y"]))

        avg = average_state_dicts(local_params, weights)
        set_model_params(global_model, avg)
        m = evaluate_global_model(global_model, X_test, y_test)
        print(f"     🟢 Acc={m['accuracy']:.4f}")

    final = evaluate_global_model(global_model, X_test, y_test)
    energy = estimate_energy_kwh(total_t)
    co2 = estimate_co2_kg(energy)
    comm = N_ROUNDS * len(clients) * MODEL_SIZE_MB

    return {
        "Scenario": "FedProx",
        "Final Accuracy": final["accuracy"],
        "Final F1-Score": final["f1"],
        "Final Recall": final["recall"],
        "Final AUC-ROC": final["auc"],
        "Total Training Time (s)": total_t,
        "Total Energy (kWh)": energy,
        "Total CO2 (kg)": co2,
        "Total Communication (MB)": comm,
        "Selected Clients": len(clients),
        "Trained Clients": len(clients),
        "Avg PScore Selected": float(np.mean(pscores)),
    }


def run_median(clients, X_test, y_test, pscores):
    print("\n🎯 Median")

    global_model = init_global_model()
    total_t = 0.0

    for r in range(1, N_ROUNDS + 1):
        print(f"  Round {r}")
        local_params = []
        for c in clients:
            cid = c["id"]
            params, t, acc = local_train(global_model, c["X"], c["y"])
            print(f"     Client {cid} acc={acc:.3f}")
            total_t += t
            local_params.append(params)

        agg = median_aggregate(local_params)
        set_model_params(global_model, agg)
        m = evaluate_global_model(global_model, X_test, y_test)
        print(f"     🟢 Acc={m['accuracy']:.4f}")

    final = evaluate_global_model(global_model, X_test, y_test)
    energy = estimate_energy_kwh(total_t)
    co2 = estimate_co2_kg(energy)
    comm = N_ROUNDS * len(clients) * MODEL_SIZE_MB

    return {
        "Scenario": "Median",
        "Final Accuracy": final["accuracy"],
        "Final F1-Score": final["f1"],
        "Final Recall": final["recall"],
        "Final AUC-ROC": final["auc"],
        "Total Training Time (s)": total_t,
        "Total Energy (kWh)": energy,
        "Total CO2 (kg)": co2,
        "Total Communication (MB)": comm,
        "Selected Clients": len(clients),
        "Trained Clients": len(clients),
        "Avg PScore Selected": float(np.mean(pscores)),
    }


def run_krum(clients, X_test, y_test, pscores):
    print("\n🎯 Krum")

    global_model = init_global_model()
    total_t = 0.0
    chosen_counts = [0] * len(clients)

    for r in range(1, N_ROUNDS + 1):
        print(f"  Round {r}")
        local_params = []

        for c in clients:
            cid = c["id"]
            params, t, acc = local_train(global_model, c["X"], c["y"])
            print(f"     Client {cid} acc={acc:.3f}")
            total_t += t
            local_params.append(params)

        agg, chosen = krum_aggregate(local_params, KRUM_F_BYZANTINE)
        if chosen >= 0:
            chosen_counts[chosen] += 1
            print(f"     🟢 Krum chose client ~{chosen}")
        else:
            print("     🟢 Fallback to FedAvg")

        set_model_params(global_model, agg)
        m = evaluate_global_model(global_model, X_test, y_test)
        print(f"     🟢 Acc={m['accuracy']:.4f}")

    final = evaluate_global_model(global_model, X_test, y_test)
    energy = estimate_energy_kwh(total_t)
    co2 = estimate_co2_kg(energy)
    comm = N_ROUNDS * len(clients) * MODEL_SIZE_MB

    return {
        "Scenario": "Krum",
        "Final Accuracy": final["accuracy"],
        "Final F1-Score": final["f1"],
        "Final Recall": final["recall"],
        "Final AUC-ROC": final["auc"],
        "Total Training Time (s)": total_t,
        "Total Energy (kWh)": energy,
        "Total CO2 (kg)": co2,
        "Total Communication (MB)": comm,
        "Selected Clients": 1,
        "Trained Clients": len(clients),
        "Avg PScore Selected": float(np.mean(pscores)),
    }


def run_power_of_choice(clients, X_test, y_test, pscores):
    print("\n🎯 Power-of-Choice (top-{} acc)".format(POC_TOP_K))

    global_model = init_global_model()
    total_t = 0.0
    last_selected = []

    for r in range(1, N_ROUNDS + 1):
        print(f"  Round {r}")
        local_params, weights, accs = [], [], []

        for c in clients:
            cid = c["id"]
            params, t, acc = local_train(global_model, c["X"], c["y"])
            print(f"     Client {cid} acc={acc:.3f}")
            total_t += t
            local_params.append(params)
            weights.append(len(c["y"]))
            accs.append(acc)

        accs = np.array(accs)
        top_idx = np.argsort(-accs)[:POC_TOP_K]
        last_selected = top_idx.tolist()
        print(f"     Selected clients: {last_selected}")

        sel_params = [local_params[i] for i in top_idx]
        sel_weights = [weights[i] for i in top_idx]

        avg = average_state_dicts(sel_params, sel_weights)
        set_model_params(global_model, avg)
        m = evaluate_global_model(global_model, X_test, y_test)
        print(f"     🟢 Acc={m['accuracy']:.4f}")

    final = evaluate_global_model(global_model, X_test, y_test)
    energy = estimate_energy_kwh(total_t)
    co2 = estimate_co2_kg(energy)
    comm = N_ROUNDS * POC_TOP_K * MODEL_SIZE_MB

    return {
        "Scenario": "Power-of-Choice",
        "Final Accuracy": final["accuracy"],
        "Final F1-Score": final["f1"],
        "Final Recall": final["recall"],
        "Final AUC-ROC": final["auc"],
        "Total Training Time (s)": total_t,
        "Total Energy (kWh)": energy,
        "Total CO2 (kg)": co2,
        "Total Communication (MB)": comm,
        "Selected Clients": len(last_selected),
        "Trained Clients": len(clients),
        "Avg PScore Selected": float(np.mean([pscores[i] for i in last_selected])),
    }


# ============================================================
#  PSCORE-WEIGHTED FEDAVG (ALL CLIENTS, NO GATE)
# ============================================================

def run_pscore_weighted_fedavg_all(clients, X_test, y_test, pscores):
    print("\n🎯 PScore-Weighted FedAvg (all clients, no gate)")

    global_model = init_global_model()
    total_t = 0.0

    ps_norm = minmax_normalize_pscores(pscores)
    f_ps = 0.5 + 1.0 * ps_norm  # [0.5, 1.5]

    for r in range(1, N_ROUNDS + 1):
        print(f"  Round {r}")
        local_params, weights = [], []

        for c in clients:
            cid = c["id"]
            params, t, acc = local_train(global_model, c["X"], c["y"])
            print(f"     Client {cid} acc={acc:.3f}")
            total_t += t

            eff_w = len(c["y"]) * float(f_ps[cid])
            local_params.append(params)
            weights.append(eff_w)

        avg = average_state_dicts(local_params, weights)
        set_model_params(global_model, avg)
        m = evaluate_global_model(global_model, X_test, y_test)
        print(f"     🟢 Acc={m['accuracy']:.4f}")

    final = evaluate_global_model(global_model, X_test, y_test)
    energy = estimate_energy_kwh(total_t)
    co2 = estimate_co2_kg(energy)
    comm = N_ROUNDS * len(clients) * MODEL_SIZE_MB

    return {
        "Scenario": "PScoreWeightedFedAvgAll",
        "Final Accuracy": final["accuracy"],
        "Final F1-Score": final["f1"],
        "Final Recall": final["recall"],
        "Final AUC-ROC": final["auc"],
        "Total Training Time (s)": total_t,
        "Total Energy (kWh)": energy,
        "Total CO2 (kg)": co2,
        "Total Communication (MB)": comm,
        "Selected Clients": len(clients),
        "Trained Clients": len(clients),
        "Avg PScore Selected": float(np.mean(pscores)),
    }


def run_pscore_weighted_fedavg_warmup(clients, X_test, y_test, pscores, warmup_rounds: int = 2):
    print("\n🎯 PScore-Weighted FedAvg (all clients, with {} warm-up rounds)".format(warmup_rounds))

    global_model = init_global_model()
    total_t = 0.0

    ps_norm = minmax_normalize_pscores(pscores)
    f_ps = 0.5 + 1.0 * ps_norm

    for r in range(1, N_ROUNDS + 1):
        mode = "warm-up" if r <= warmup_rounds else "pscore-weighted"
        print(f"  Round {r} [{mode}]")

        local_params, weights = [], []
        for c in clients:
            cid = c["id"]
            params, t, acc = local_train(global_model, c["X"], c["y"])
            print(f"     Client {cid} acc={acc:.3f}")
            total_t += t

            if r <= warmup_rounds:
                eff_w = len(c["y"])
            else:
                eff_w = len(c["y"]) * float(f_ps[cid])

            local_params.append(params)
            weights.append(eff_w)

        avg = average_state_dicts(local_params, weights)
        set_model_params(global_model, avg)
        m = evaluate_global_model(global_model, X_test, y_test)
        print(f"     🟢 Acc={m['accuracy']:.4f}")

    final = evaluate_global_model(global_model, X_test, y_test)
    energy = estimate_energy_kwh(total_t)
    co2 = estimate_co2_kg(energy)
    comm = N_ROUNDS * len(clients) * MODEL_SIZE_MB

    return {
        "Scenario": "PScoreWeightedFedAvgWarmup",
        "Final Accuracy": final["accuracy"],
        "Final F1-Score": final["f1"],
        "Final Recall": final["recall"],
        "Final AUC-ROC": final["auc"],
        "Total Training Time (s)": total_t,
        "Total Energy (kWh)": energy,
        "Total CO2 (kg)": co2,
        "Total Communication (MB)": comm,
        "Selected Clients": len(clients),
        "Trained Clients": len(clients),
        "Avg PScore Selected": float(np.mean(pscores)),
    }


# ============================================================
#  STATIC PSCORE GATE (HARD GATE + PSCORE-WEIGHTED)
# ============================================================

def run_static_pscore_gate(clients, X_test, y_test, pscores, t_gov: float):
    print("\n🎯 Static PScore Gate (hard gate + PScore-weighted FedAvg)")

    global_model = init_global_model()
    accepted_ids = [i for i, s in enumerate(pscores) if s >= t_gov]
    print(f"   Threshold={t_gov:.3f} → accepted: {accepted_ids}")

    active = [c for c in clients if c["id"] in accepted_ids]
    total_t = 0.0

    if len(active) == 0:
        print("   ⚠️ No clients above threshold; falling back to FedAvg.")
        return run_fedavg(clients, X_test, y_test, pscores)

    ps_norm = minmax_normalize_pscores(pscores)

    for r in range(1, N_ROUNDS + 1):
        print(f"  Round {r}")
        local_params, weights = [], []

        for c in active:
            cid = c["id"]
            params, t, acc = local_train(global_model, c["X"], c["y"])
            print(f"     Client {cid} acc={acc:.3f}")
            total_t += t

            eff_w = len(c["y"]) * (0.5 + 1.0 * ps_norm[cid])
            local_params.append(params)
            weights.append(eff_w)

        avg = average_state_dicts(local_params, weights)
        set_model_params(global_model, avg)
        m = evaluate_global_model(global_model, X_test, y_test)
        print(f"     🟢 Acc={m['accuracy']:.4f}")

    final = evaluate_global_model(global_model, X_test, y_test)
    energy = estimate_energy_kwh(total_t)
    co2 = estimate_co2_kg(energy)
    comm = N_ROUNDS * len(active) * MODEL_SIZE_MB

    return {
        "Scenario": "StaticPScoreGate",
        "Final Accuracy": final["accuracy"],
        "Final F1-Score": final["f1"],
        "Final Recall": final["recall"],
        "Final AUC-ROC": final["auc"],
        "Total Training Time (s)": total_t,
        "Total Energy (kWh)": energy,
        "Total CO2 (kg)": co2,
        "Total Communication (MB)": comm,
        "Selected Clients": len(active),
        "Trained Clients": len(active),
        "Avg PScore Selected": float(np.mean([pscores[i] for i in accepted_ids])),
    }


# ============================================================
#  DYNAMIC PSCORE-v3 (SOFT PENALTY, NO FILTERING)
# ============================================================

def run_dynamic_pscore_v3(clients, X_test, y_test, pscores, t_gov: float):
    print("\n🎯 Dynamic PScore-v3 (warm-up + soft down-weighting)")

    global_model = init_global_model()
    total_t = 0.0

    ps_norm = minmax_normalize_pscores(pscores)
    high_ids = [i for i, s in enumerate(pscores) if s >= t_gov]
    low_ids = [i for i, s in enumerate(pscores) if s < t_gov]

    print(f"   Static threshold={t_gov:.3f}")
    print(f"   High-provenance clients: {high_ids}")
    print(f"   Low-provenance clients (penalized): {low_ids}")

    for r in range(1, N_ROUNDS + 1):
        mode = "warm-up" if r <= DYN_WARMUP_ROUNDS else "gov-weighted"
        print(f"  Round {r} [{mode}]")

        local_params, weights = [], []
        for c in clients:
            cid = c["id"]
            params, t, acc = local_train(global_model, c["X"], c["y"])
            print(f"     Client {cid} acc={acc:.3f}")
            total_t += t

            if r <= DYN_WARMUP_ROUNDS:
                eff_w = len(c["y"])
            else:
                norm_p = ps_norm[cid]
                if cid in low_ids:
                    norm_p *= DYN_LOW_PENALTY
                g_factor = DYN_ALPHA_PROV * norm_p + (1.0 - DYN_ALPHA_PROV)
                eff_w = len(c["y"]) * max(g_factor, 1e-3)

            local_params.append(params)
            weights.append(eff_w)

        avg = average_state_dicts(local_params, weights)
        set_model_params(global_model, avg)
        m = evaluate_global_model(global_model, X_test, y_test)
        print(f"     🟢 Acc={m['accuracy']:.4f}")

    final = evaluate_global_model(global_model, X_test, y_test)
    energy = estimate_energy_kwh(total_t)
    co2 = estimate_co2_kg(energy)
    comm = N_ROUNDS * len(clients) * MODEL_SIZE_MB

    return {
        "Scenario": "DynamicPScoreV3",
        "Final Accuracy": final["accuracy"],
        "Final F1-Score": final["f1"],
        "Final Recall": final["recall"],
        "Final AUC-ROC": final["auc"],
        "Total Training Time (s)": total_t,
        "Total Energy (kWh)": energy,
        "Total CO2 (kg)": co2,
        "Total Communication (MB)": comm,
        "Selected Clients": len(clients),
        "Trained Clients": len(clients),
        "Avg PScore Selected": float(np.mean([pscores[i] for i in high_ids])),
    }


# ============================================================
#  MAIN PIPELINE
# ============================================================

def main():
    set_global_seed()

    # 1) Load dataset
    X_train, X_test, y_train, y_test = load_fmnist()

    # 2) Non-IID client splits
    clients = create_dirichlet_clients(
        X_train,
        y_train,
        n_clients=N_CLIENTS,
        alpha=ALPHA_DIRICHLET,
    )

    # 3) PScore vector + governance threshold
    pscores = get_static_pscores(N_CLIENTS)
    print("📊 Static PScore values:")
    for cid, s in enumerate(pscores):
        print(f"   Client {cid}: {s:.2f}")
    print()

    t_gov = compute_governance_threshold(pscores, GOVERNANCE_PERCENTILE)
    print(f"🔒 Static governance threshold from {GOVERNANCE_PERCENTILE:.0f}th percentile: {t_gov:.3f}\n")

    # 4) Run baselines + governance variants (medium pipeline)
    all_results = []

    # No-governance baselines
    all_results.append(run_fedavg(clients, X_test, y_test, pscores))
    all_results.append(run_fedprox(clients, X_test, y_test, pscores, mu=0.01))

    # Robust FL
    all_results.append(run_median(clients, X_test, y_test, pscores))
    all_results.append(run_krum(clients, X_test, y_test, pscores))

    # Data-driven selection
    all_results.append(run_power_of_choice(clients, X_test, y_test, pscores))

    # Governance-aware variants
    all_results.append(run_pscore_weighted_fedavg_all(clients, X_test, y_test, pscores))
    all_results.append(run_pscore_weighted_fedavg_warmup(clients, X_test, y_test, pscores, warmup_rounds=DYN_WARMUP_ROUNDS))
    all_results.append(run_static_pscore_gate(clients, X_test, y_test, pscores, t_gov))
    all_results.append(run_dynamic_pscore_v3(clients, X_test, y_test, pscores, t_gov))

    # 5) Final summary
    print("\n" + "=" * 100)
    print("📊 FINAL RESULTS — Fashion-MNIST (Governance + Robust FL, Medium Pipeline)")
    print("=" * 100)

    df = pd.DataFrame(all_results)

    cols_order = [
        "Scenario",
        "Final Accuracy",
        "Final F1-Score",
        "Final Recall",
        "Final AUC-ROC",
        "Total Training Time (s)",
        "Total Energy (kWh)",
        "Total CO2 (kg)",
        "Total Communication (MB)",
        "Selected Clients",
        "Trained Clients",
        "Avg PScore Selected",
    ]
    cols_order = [c for c in cols_order if c in df.columns]
    df = df[cols_order]

    if "Final Accuracy" in df.columns:
        df = df.sort_values(by=["Final Accuracy", "Scenario"], ascending=[False, True])

    print(df.to_string(index=False, float_format=lambda x: f"{x:.4f}"))

    out_csv = "fmnist_governance_results_medium_pipeline.csv"
    df.to_csv(out_csv, index=False)
    print(f"\n💾 Saved results to '{out_csv}'")


if __name__ == "__main__":
    main()


📊 Loading Fashion-MNIST...
✅ F-MNIST loaded: train=60000, test=10000
🔧 Creating Non-IID clients (Dirichlet α=0.5)
   Client 0: 11403 samples → {np.int64(0): np.int64(379), np.int64(1): np.int64(20), np.int64(2): np.int64(13), np.int64(3): np.int64(1605), np.int64(4): np.int64(752), np.int64(5): np.int64(4718), np.int64(6): np.int64(1781), np.int64(7): np.int64(18), np.int64(8): np.int64(3), np.int64(9): np.int64(2114)}
   Client 1: 13670 samples → {np.int64(0): np.int64(150), np.int64(1): np.int64(44), np.int64(2): np.int64(3), np.int64(3): np.int64(2872), np.int64(4): np.int64(271), np.int64(5): np.int64(185), np.int64(6): np.int64(2334), np.int64(7): np.int64(2548), np.int64(8): np.int64(5106), np.int64(9): np.int64(157)}
   Client 2: 13192 samples → {np.int64(0): np.int64(284), np.int64(1): np.int64(3953), np.int64(2): np.int64(1132), np.int64(3): np.int64(16), np.int64(4): np.int64(2713), np.int64(5): np.int64(95), np.int64(6): np.int64(1514), np.int64(7): np.int64(1401), np.int64(